# Cybersecurity advanced
## Lab 07: Backups


Links to Syllabus

- [Theoretical](https://hogenttin.github.io/cybersecurity-advanced/lesson-7/theory/)
- [Practical](https://hogenttin.github.io/cybersecurity-advanced/lesson-7/lab/)

BORG KEY:

```BORG_KEY 018f8b7276ac043e194a67ffed9fedd5a6244c5248645e3db9310a79c5d9aa48
hqlhbGdvcml0aG2mc2hhMjU2pGRhdGHaAN4opkPkw1kz9fgPsoeNQvJwoBlBescuvxjG3R
FCIlOdiGua6hBN3kNAszyQqwV6/hGTAoronMg7hKIKn5VsgSB0i3STrH1rn91Dtp/M8a/S
8klZuH1BbYAOEOxqoolLCb9CQmwHKrz7vcg0HQBbvzORdYclneLaznqogTTVQEWsheZCNh
58CEeOTjDgb8sjyfrwQQVS41tNADXOhltpxA4C/6ZPxENEhIyHqYy2YFxzVcaYilDrDmoZ
GeJgbgGfxzA4oeOP7ic5LotEcTooJgveBYeslCQtGDiHbRKFCTikaGFzaNoAICB9oKqB+G
Ifut4R/YiHcbcbGxwHdFYnorXdIRNFBa2eqml0ZXJhdGlvbnPOAAGGoKRzYWx02gAgpcGn
U3G1ZuwdW5edcoAksvOsPtJNuq2LNyASk/l2QwCndmVyc2lvbgE=```

To install borg:

WEBSERVER
```bash
mkdir important-files
cd important-files

curl --remote-name-all
https://video.blender.org/download/videos/bf1f3fb5-b119-4f9f-9930-8e20e892b898-720.mp4 https://www.gutenberg.org/ebooks/100.txt.utf-8 https://www.gutenberg.org/ebooks/996.txt.utf-8 https://upload.wikimedia.org/wikipedia/commons/4/40/Toreador_song_cleaned.ogg

mv 100.txt.utf-8 100.txt
mv 996.txt.utf-8 996.txt
ll

sudo dnf install epel-release -y
sudo dnf search borgbackup
sudo dnf install borgbackup -y

borg init --encryption=repokey vagrant@172.30.0.15:/home/vagrant/backups

echo "export BORG_PASSPHRASE=vagrant" >> ~/.bashrc
source ~/.bashrc
echo $BORG_PASSPHRASE
```

DATABASE
```bash
mkdir backups

sudo dnf install epel-release -y
sudo dnf search borgbackup
sudo dnf install borgbackup -y

borg key export /home/agrant/backups/
cat config
```

BACKUPS
- Web
```bash
#Create
borg create vagrant@172.30.0.15:/home/vagrant/backups::first /home/vagrant/important-files
#Show
borg info vagrant@172.30.0.15:/home/vagrant/backups
borg list vagrant@172.30.0.15:/home/vagrant/backups
```

RESTORE TO BACKUP
- Web
```bash
borg extract vagrant@172.30.0.15:/home/vagrant/backups::first /home/vagrant --strip-components=2
ll important-files/
```

# Service

CREATE SCRIPT
```bash
mkdir -p /home/vagrant/scripts
vi backup_script.sh

    #!/bin/bash

    export BORG_PASSPHRASE="vagrant"
    borg create
    vagrant@172.30.0.15:/home/vagrant/backups::'{now:%Y-%m-%d}' /home/vagrant/important-files
    borg prune -v --prefix '{now:%Y-%m-%d}' --keep-within=5m --keep-hourly=24 --keep-daily=7 --keep-weekly=4


sudo su
cd /etc/systemd/system/
vi borg-backup.timer

    [Unit]
    Description=BorgBackup 5min timer

    [Timer]
    OnCalendar=*:0/5
    Unit=borg-backup.service

    [Install]
    WantedBy=timers.target

vi borg-backup.service

    [Unit]
    Description=Borg Backup Service

    [Service]
    Type=simple
    Environment="BORG_REPO=vagrant@172.30.0.15:/home/vagrant/backups"
    Environment="BORG_PASSPHRASE=vagrant"
    ExecStart=/bin/bash /home/vagrant/scripts/backup_script.sh
    User=vagrant
    Group=vagrant
    Restart=on-failure

    [Install]
    WantedBy=multi-user.target

sudo systemctl daemon-reload
sudo systemctl enable borg-backup.timer
sudo systemctl start borg-backup.service
sudo systemctl start borg-backup.timer
sudo systemctl list-timers
```

# Demo

BACKUPS
- Web
```bash
#Create
borg create vagrant@172.30.0.15:/home/vagrant/backups::first /home/vagrant/important-files
#Show
borg info vagrant@172.30.0.15:/home/vagrant/backups
borg list vagrant@172.30.0.15:/home/vagrant/backups
```

RESTORE TO BACKUP
- Web
```bash
borg extract vagrant@172.30.0.15:/home/vagrant/backups::first /home/vagrant --strip-components=2
ll important-files/
```